# Implementation of conventional Warfare Game modeling framework

In [ ]:
#!pip install pygame
#!pip install mesa

To begin, we build a simple agent-based model using the Mesa framework in Python for a conventional battle scene. For this, we first call the following basic libraries:

In [ ]:
import pygame
import matplotlib.pyplot as plt
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from tqdm import tqdm
import time

As can be seen in the lines of the above program, the pygame library is called for the visual representation of the war game, and then the Mesa library is called for the implementation of the agent-based model. Matplotlib is also used to plot the number of casualties (defeated actors) over time. The tqdm library is also used to display the progress bar during the simulation.

To make it easier to use the color scheme, we have defined the color for the actors:

In [ ]:
# Define colors for the actors
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

**Main program body:**

In the following, we define the general framework of the program in two classes, actor and Warfare, with the object-oriented programming model, and with the agent and model objects, respectively:

This is a very basic example that creates a number of agents (in this case actors), places them on a grid, and runs their step function at each simulation iteration. The step function is where you can implement the player's strategy based on the abilities and rules of the game. The Actor class represents an individual actor in the simulation, such as a military unit or a state or a country. Each player has different characteristics, including a unique ID, the ability to randomly choose from among players, the type of player's strategy, and finally the color assigned to it for display on the war game screen. It is important to mention here that although the selection of the actor is written randomly using the random method, it is possible to limit the possible choices to the input conditions by the programmer, and therefore the degree of randomness can be controlled. Was. The capability property shows the actor's military ability and a random value between 1 and 10 is assigned to it during initialization. The strategy feature shows the actor's current strategy, which is set to "attack" for all actors, but it is possible to define the strategy in multiple ways based on the needs of the performers and create the possibility of choosing each actor with a different strategy. The step method of the Actor class implements the actor's strategy. If the strategy is "Attack", the chosen player selects a random neighboring player as a target and deals damage based on its ability.
. We have tried to explain all the lines of code in the program. As can be seen in the class below, first the actors are randomly selected and then to implement the strategy of the actors, after defining the step, we proceed to define the attack strategy. The capability of each actor is chosen with a random number, but the strategy is defined by default in attack mode, which assumes that this attack is against a random target of the defending actor class, which is located in the neighborhood of the attacker. If in this attack, the ability of the target drops to zero or less, the target is removed from the game.

In [ ]:
class Actor(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.capability = self.random.randint(1, 10) # Assign random capabilities to each actor
        self.strategy = "attack" # Set strategy to "attack" for all actors
        self.color = None

    def step(self):
        # Implement actor's strategy
        if self.strategy == "attack":
            # Find neighboring cells with other actors
            neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
            targets = [agent for agent in neighbors if isinstance(agent, Actor) and agent.unique_id != self.unique_id]
            if targets:
                # Attack a random target
                target = self.random.choice(targets)
                damage = self.random.randint(1, self.capability)
                target.capability -= damage
                # Check if the target is defeated
                if target.capability <= 0:
                    self.model.grid.remove_agent(target)
                    self.model.schedule.remove(target)

class Warfare(Model):
    def __init__(self, num_actors, width, height):
        self.num_actors = num_actors
        self.grid = MultiGrid(width, height, torus=True)
        self.schedule = RandomActivation(self)
        self.casualties = []
        
        # Create actors
        for i in range(self.num_actors):
            actor = Actor(i, self)
            actor.color = RED if i % 2 == 0 else BLUE # Assign colors to actors based on their unique_id
            self.schedule.add(actor)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(actor, (x, y))
        
        self.running = True
        self.winner = None

    def step(self):
        self.schedule.step()
        # Check for victory or defeat conditions
        num_alive = sum([1 for agent in self.schedule.agents if isinstance(agent, Actor)])
        if num_alive == 0:
            self.running = False
            self.winner = "Draw"
        elif num_alive == 1:
            self.running = False
            self.winner = "Actor " + str([agent for agent in self.schedule.agents if isinstance(agent, Actor)][0].unique_id)
        self.casualties.append(self.num_actors - len(self.schedule.agents))
    
    def get_casualties(self):
        return self.num_actors - len(self.schedule.agents)

if __name__ == "__main__":
    # Initialize Pygame
    pygame.init()
    screen = pygame.display.set_mode((500, 500))
    pygame.display.set_caption("Warfare Simulation")

    # Create model
    model = Warfare(num_actors= 32, width= 100, height= 100)

    # Main simulation loop
    with tqdm(total=100) as pbar:
        while model.running:
            # Handle events
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    model.running = False

            # Clear the screen
            screen.fill((255, 255, 255))

            # Draw actors
            for actor in model.schedule.agents:
                x, y = actor.pos
                pygame.draw.rect(screen, actor.color, (x*10, y*10, 10, 10))

            # Update the model
            model.step()

            # Update the screen
            pygame.display.flip()

            # Update the progress bar
            pbar.update(1)

            # Optional delay to slow down the simulation
            # time.sleep(0.1)

    # Print the outcome of the simulation
    if model.winner:
        print("The winner is:", model.winner)
    else:
        print("The simulation ended in a draw")

    print("Casualties:", model.get_casualties())

    # Plot the casualties over time
    plt.plot(model.casualties)
    plt.title("Casualties over time")
    plt.xlabel("Iteration")
    plt.ylabel("Casualties")
    plt.show()

    # Create a report on the simulation
    report = f"""Simulation Outcome:
    Winner: {model.winner or 'Draw'}
    Casualties: {model.get_casualties()}
    """

    print(report)

    # Quit Pygame
    pygame.quit()


In the above class, the actors try to find their neighbors among the actors and randomly attack the targets. Due to the fact that this program provides a general framework, we have refrained from stating the specific conditions of the goals, actors and the scene of the game. Obviously, according to the practical problem, each of these items can be added to the above class. Further, the Warfare class is also implemented as follows. In this class, first, the length and width of the battle scene is determined and based on the number of selected actors, blue and red colors are assigned to the defender and attacker, respectively, and the actors are randomly distributed on the screen with small squares. Then, in each step of the simulation, the conditions of victory or defeat are established and the number of killed, victorious and playing players is determined. The Warfare class represents the simulation model. This model includes features such as num_actors, grid, schedule and casualties. The num_actors property determines the number of actors in the simulation. The grid feature shows the multi-grid space where the actors are placed and interact with each other. The schedule attribute manages the activation order of the actors. The casualty property is a list that keeps track of the number of actors (casualties) in each iteration of the simulation. The init method of the Warfare class initializes the simulation model. Creates a specified number of actors, assigns colors based on their unique id, randomly places them in the grid, and sets the running property to true. The step method of the Warfare class advances the simulation by one step.

All players have the same strategy, which is to attack other players. The simulation proceeds in discrete time steps, and during each time step, each actor randomly chooses a neighboring actor to attack and randomly deals damage to it based on its capabilities. If an actor's ability reaches zero or less, he is removed from the network. The simulation continues until only one player remains or all players fail.
At the end of the simulation, the code shows the result (winner or if the simulation ends with a draw) and the number of casualties. It also produces a text report of the simulation containing similar information.